In [1]:
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from datasets import Dataset
import numpy as np
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from transformers import AutoConfig, RobertaModel, RobertaForSequenceClassification
from transformers.modeling_outputs import SequenceClassifierOutput
import pandas as pd,os
import torch
from statistics import mode
from sklearn.metrics import confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer
from datasets import Dataset, load_metric
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import pickle
from transformers import RobertaConfig, RobertaModel
import torch
from torch import nn
import torch
from torch import nn
from transformers import AutoConfig, RobertaModel, RobertaForSequenceClassification
from transformers.modeling_outputs import SequenceClassifierOutput
from typing import Optional, Union, Tuple
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
from transformers import EarlyStoppingCallback,AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from datasets import Dataset
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from textstat.textstat import textstatistics
import pandas as pd
import language_tool_python
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score, roc_auc_score, confusion_matrix
import numpy as np
from itertools import combinations
from sklearn.ensemble import RandomForestClassifier

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config, num_extra_dims):
        super().__init__()
        total_dims = config.hidden_size+num_extra_dims
        self.dense = nn.Linear(total_dims, total_dims)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(total_dims, config.num_labels)

    def forward(self, features, **kwargs):
        x = self.dropout(features)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class CustomSequenceClassification(RobertaForSequenceClassification):

    def __init__(self, config, num_extra_dims):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        # might need to rename this depending on the model
        self.roberta =  RobertaModel(config)
        self.classifier = ClassificationHead(config, num_extra_dims)

        # Initialize weights and apply final processing
        self.post_init()

    
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        extra_data: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # sequence_output will be (batch_size, seq_length, hidden_size)
        sequence_output = outputs[0]

        # additional data should be (batch_size, num_extra_dims)
        cls_embedding = sequence_output[:, 0, :]

        output = torch.cat((cls_embedding, extra_data), dim=-1)

        logits = self.classifier(output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = nn.MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = nn.BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [4]:

MODEL1 = 'bert-base-uncased'
MODEL2 = 'Hello-SimpleAI/chatgpt-detector-roberta'
MODEL3 = 'roberta-base-openai-detector'
MODEL4 = 'roberta-base'
MODEL5 = 'distilbert-base-uncased'
MODEL6 = 'google/electra-base-discriminator'

MODEL8='multidomain-roberta-base'
MODEL9='multidomain-extended-roberta-base'

MODEL_PATH1='SavedModels_A/optimized-bert-base-uncased-22.5k'
MODEL_PATH2='SavedModels_A/optimized-chatgpt-detector-roberta-17.5k'
MODEL_PATH3='SavedModels_A/optimized-roberta-base-openai-detector-12k'
MODEL_PATH4='SavedModels_A/optimized-roberta-base-0.5k'
MODEL_PATH5='SavedModels_A/optimized-distilbert-base-uncased-15k'
MODEL_PATH6='SavedModels_A/optimized-electra-base-discriminator-6k'

MODEL_PATH8='SavedModels_A/optimized-multidomain-roberta-base-1k'
MODEL_PATH9='SavedModels_A/optimized-multidomain-extended-robera-base-0.8k'

'''Preparing data'''

df = pd.read_json('datasets/subtaskA_test_monolingual.jsonl', lines=True)

# df = df.rename(columns={'source': 'language'})
# non_language_sources = ['wikihow', 'wikipedia', 'reddit', 'arxiv', 'peerread']
# df['language'] = df['language'].replace(non_language_sources, 'english')

print(f'Original dataset')
print(df.info())

# print(f'''\n{df['label'].value_counts()}''')
# print(f'''\n{df['model'].value_counts()}''')
# print(f'''\n{df['language'].value_counts()}''')

# df = df[['text', 'label']]
test_df=df

test_texts = test_df['text'].tolist()

def getPrediction(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    id2label = {0: "human", 1: "machine"}
    label2id = {"human": 0, "machine": 1}
    model = AutoModelForSequenceClassification.from_pretrained(
        model_path, num_labels=len(label2id), id2label=id2label, label2id=label2id
        )

    test_dataset = Dataset.from_pandas(test_df)

    def preprocess_function(examples, **fn_kwargs):
        return fn_kwargs['tokenizer'](examples["text"], truncation=True)

    tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # create Trainer
    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        data_collator=data_collator,
        # compute_metrics=compute_metrics,
    )
    # get logits from predictions and evaluate results using classification report
    predictions = trainer.predict(tokenized_test_dataset)
    preds = np.argmax(predictions.predictions, axis=-1)
    probs = softmax(predictions.predictions, axis=-1)
    label_specific_probs = probs[:, 1]  # This extracts the probability for label 1
    
    return list(preds),list(label_specific_probs)

def getPredictionMultidomain(model_path,num_extra_dims,test_data):
    config = RobertaConfig.from_pretrained(model_path)
    model = CustomSequenceClassification(config, num_extra_dims)
    model.load_state_dict(torch.load(model_path+'/pytorch_model.bin'))
    trainer = Trainer(model=model)
    # get logits from predictions and evaluate results using classification report
    predictions = trainer.predict(test_data)
    preds = np.argmax(predictions.predictions, axis=-1)
    probs = softmax(predictions.predictions, axis=-1)
    label_specific_probs = probs[:, 1]  # This extracts the probability for label 1
    
    return list(preds),list(label_specific_probs)

print(f'\nProcessing with {MODEL1}')
labels1,scores1=getPrediction(MODEL_PATH1)

print(f'\nProcessing with {MODEL2}')
labels2,scores2=getPrediction(MODEL_PATH2)

print(f'\nProcessing with {MODEL3}')
labels3,scores3=getPrediction(MODEL_PATH3)

print(f'\nProcessing with {MODEL4}')
labels4,scores4=getPrediction(MODEL_PATH4)

print(f'\nProcessing with {MODEL5}')
labels5,scores5=getPrediction(MODEL_PATH5)

print(f'\nProcessing with {MODEL6}')
labels6,scores6=getPrediction(MODEL_PATH6)


Original dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34272 entries, 0 to 34271
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    34272 non-null  object
 1   id      34272 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 535.6+ KB
None

Processing with bert-base-uncased


Map: 100%|██████████| 34272/34272 [00:09<00:00, 3704.91 examples/s]
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 4284/4284 [02:49<00:00, 25.26it/s]



Processing with Hello-SimpleAI/chatgpt-detector-roberta


Map: 100%|██████████| 34272/34272 [00:08<00:00, 4148.24 examples/s]
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 4284/4284 [02:47<00:00, 25.59it/s]



Processing with roberta-base-openai-detector


...


Processing with distilbert-base-uncased


Map: 100%|██████████| 34272/34272 [00:11<00:00, 2974.55 examples/s]
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 4284/4284 [01:29<00:00, 47.68it/s]



Processing with google/electra-base-discriminator


Map: 100%|██████████| 34272/34272 [00:13<00:00, 2590.25 examples/s]
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 4284/4284 [02:49<00:00, 25.29it/s]


KeyError: "['text', 'label'] not in index"

In [6]:
df_multidomain_extended_extraData=pd.read_csv('datasets/features_df_test_A.csv')
reduced_columns=['word_count','avg_sentence_length','avg_word_length','gunning_fog_index','grammatical_errors']
df_multidomain_extraData=df_multidomain_extended_extraData[reduced_columns]

# df_multidomain_extraData=df_multidomain.drop(['label'],axis=1)
# df_multidomain_extraData=df_multidomain_extraData.drop(['text'],axis=1)

# df_multidomain_extended_extraData=df_multidomain_extended.drop(['label'],axis=1)
# df_multidomain_extended_extraData=df_multidomain_extended_extraData.drop(['text'],axis=1)

ds_test = Dataset.from_dict({
        "text": test_df['text'].tolist(), 
        "extra_data": df_multidomain_extraData.values.tolist(),
        # "labels": df_multidomain['label'].tolist()
    })

ds_test_extended = Dataset.from_dict({
        "text": test_df['text'].tolist(), 
        "extra_data": df_multidomain_extended_extraData.values.tolist(),
        # "labels": df_multidomain_extended['label'].tolist()
    })

tokenizer = AutoTokenizer.from_pretrained('roberta-base')
tokenized_ds_test = ds_test.map(lambda x: tokenizer(x["text"], padding='max_length', truncation=True))
tokenized_ds_test = tokenized_ds_test.map(lambda x: {'extra_data': x['extra_data']})
tokenized_ds_test_extended = ds_test_extended.map(lambda x: tokenizer(x["text"], padding='max_length', truncation=True))
tokenized_ds_test_extended = tokenized_ds_test_extended.map(lambda x: {'extra_data': x['extra_data']})

print(f'\nProcessing with {MODEL8}')
labels8,scores8=getPredictionMultidomain(MODEL_PATH8,5,tokenized_ds_test)

print(f'\nProcessing with {MODEL9}')
labels9,scores9=getPredictionMultidomain(MODEL_PATH9,9,tokenized_ds_test_extended)

Map: 100%|██████████| 34272/34272 [00:01<00:00, 29362.81 examples/s]



Processing with multidomain-roberta-base


100%|██████████| 4284/4284 [02:49<00:00, 25.27it/s]



Processing with multidomain-extended-roberta-base


100%|██████████| 4284/4284 [02:47<00:00, 25.61it/s]


In [7]:
feature_df=pd.read_csv('datasets/features_df_test_A.csv')
best_features=['word_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors']
feature_df=feature_df[best_features]

with open('SavedModels_A/optimized-rf-features-20k.pkl', 'rb') as file:
    randomForest = pickle.load(file)
MODEL7='Features Model'
labels7=randomForest.predict(feature_df)
probabilities = randomForest.predict_proba(feature_df)
scores7 = probabilities.max(axis=1)

In [8]:
def ensemble_methods(df, models):
    
    majority_labels = []

    for index, row in df.iterrows():
        label_counts = {0: 0, 1: 0}
        score_sums = {0: 0.0, 1: 0.0}

        for i, model in enumerate(models):
            label = row[f'Labels_{model}']
            score = row[f'Scores_{model}']
            label_counts[label] += 1
            score_sums[label] += score

        # Majority Voting
        majority_label = 0 if label_counts[0] > label_counts[1] else 1
        majority_labels.append(majority_label)
        
    return majority_labels

models = [MODEL3, MODEL4, MODEL6, MODEL7, MODEL8, MODEL9]

df_reduced = pd.DataFrame({
    f'Labels_{MODEL3}': labels3,
    f'Scores_{MODEL3}': scores3,
    f'Labels_{MODEL4}': labels4,
    f'Scores_{MODEL4}': scores4,
    f'Labels_{MODEL6}': labels6,
    f'Scores_{MODEL6}': scores6,
    f'Labels_{MODEL7}': labels7,
    f'Scores_{MODEL7}': scores7,
    f'Labels_{MODEL8}': labels8,
    f'Scores_{MODEL8}': scores8,
    f'Labels_{MODEL9}': labels9,
    f'Scores_{MODEL9}': scores9,
})

ensemble_results = ensemble_methods(df_reduced, models)

In [15]:
len(ensemble_results),len(df['id'])

new_df = pd.DataFrame({
    'id': df['id'],
    'label': ensemble_results
})

# Exporting to a jsonl file
new_df.to_json('datasets/subtaskA_predictions.jsonl', orient='records', lines=True)

In [16]:
new_df

,id,label
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
34267,34267,1
34268,34268,1
34269,34269,1
34270,34270,0
